In [126]:
import pandas as pd
from pymongo import MongoClient
from pandas.io.json import json_normalize

In [127]:
client = MongoClient('mongodb://localhost:27017/')
db = client.companies
collection = db.companies

In [128]:
# FILTER {$and: [ {deadpooled_year:null}, {founded_year: {$gt:2003}}, {offices: { $ne: [] } }, {$or: [{category_code:'games_video'},{category_code:'software'}] }]}
# PROJECT {name:1 , category_code:1, number_of_employees:1, founded_year:1, "offices.city":1, "offices.country_code":1, "offices.latitude":1, "offices.longitude":1}
# SORT {number_of_employees:-1}

In [129]:
def query_to_db():
    cursor = collection.find({'$and': 
                        [ {'deadpooled_year':None}, 
                          {'founded_year': {'$gt':2003}},
                          {'offices': { '$ne': [] } },
                          {'$or': [{'category_code':'games_video'},
                                   {'category_code':'software'}] 
                        }]}
                        , {'name':1 , 'category_code':1, 'number_of_employees':1, 'founded_year':1, 
                           "offices.city":1, "offices.country_code":1, "offices.latitude":1, "offices.longitude":1}
                        ).sort('number_of_employees', -1)
    return pd.DataFrame(cursor)

In [130]:
def get_country(element):
    return element[0]['country_code']

def get_city(element):
    return element[0]['city']

def get_lat(element):
    return element[0]['latitude']

def get_long(element):
    return element[0]['longitude']



def get_columns(data):
    data['country']= data['offices'].apply(get_country)
    data['city']= data['offices'].apply(get_city)
    data['lat']= data['offices'].apply(get_lat)
    data['long']= data['offices'].apply(get_long)
    return data
    


In [134]:
def clean_cols_nans(data):
    data= data.drop(['_id', 'offices'], axis=1)
    data= data.dropna(subset=['lat', 'long'])
    return data

In [135]:
# Main

df= query_to_db()
df_selected= get_columns(df)
df_clean= clean_cols_nans(df_selected)

In [136]:
df_clean.head()

,category_code,founded_year,name,number_of_employees,country,city,lat,long
0,software,2004,UCWeb,1700.0,CHN,Guangzhou,23.133754,113.385331
1,software,2006,Veeam Software,1500.0,USA,Columbus,40.088052,-83.088566
5,software,2006,HubSpot,650.0,USA,Cambridge,42.375392,-71.118487
6,games_video,2006,Kabam,650.0,USA,San Francisco,37.394456,-122.078531
7,software,2005,Bazaarvoice,600.0,USA,Austin,30.407545,-97.717667


In [137]:
df_clean.to_json('data_to_plot_mongo.json', orient='records')